Zero shot Audio classifcation

In [ ]:
!pip install datasets
!pip install soundfile
!pip install librosa

In [ ]:
## get dataset from hugging face hub
## esc50 is a labelled dataset of 5 sec environmental sounds- animals, humans, urban sounds

from datasets import load_dataset

dataset = load_dataset("ashraq/esc50", split = "train[0:10]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
## instance from dataset
idx = 6
sample = dataset[idx]
sample

{'filename': '1-101336-A-30.wav',
 'fold': 1,
 'target': 30,
 'category': 'door_wood_knock',
 'esc10': False,
 'src_file': 101336,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0.00012207, 0.00018311, 0.00012207, ..., 0.        , 0.        ,
         0.        ]),
  'sampling_rate': 44100}}

In [ ]:
## listening the sample
from IPython.display import Audio as IPythonAudio

IPythonAudio(sample["audio"]["array"], rate = sample["audio"]["sampling_rate"])

In [ ]:
##import pipeline
from transformers import pipeline

zs_classifier = pipeline(
    task = "zero-shot-audio-classification",
    model = "laion/clap-htsat-unfused"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/5.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

### Capturing sound and Sampling
Sound wave is a continus signal when we capture sound through microphone it is captured as an analog signal converted to an electrical signal and then we sample the signal with a saompleing rate to get the digital represnetation.

Sampling: measurement of the value of a continous signal at fixed time steps.

Sampling rate (Hz): number of samples taken per second.

* 8000 Hz or 8kHz: sampling rate for telephone.

* 16000 Hz or 16kHz: sampling rate for human speech recording.

* 192000 Hz or 192kHz: high-res audio sampling rate.


So, a 5 second audio sampled at 8kHz will give = 5*8000 = 40000 signal values.

A transformers model trained on 16 kHz when fed an array or signal values of 960,000 will be like

time (secs) * 16kHz = 960000

time(secs) = 60 second

so array of 960,000 values when fed to transformers model trained on 16 kHz will be equivalent to a 60-second recording.

In [ ]:
#audio example to be classified should have sampling rate that the model expects

zs_classifier.feature_extractor.sampling_rate

#model sampling rate - 48kHz
#5 second audio samples to this model

print(f"input array or samples to be fed to the model: {5 * 48000}")

input array or samples to be fed to the model: 240000


In [ ]:
##sample rate of the sudio sample
sample["audio"]["sampling_rate"]
#not much difference as the models sampling rate and audio sample sampling rate are kind of similar so this might work here

44100

In [ ]:
#cast dataset to correct sampling rate

from datasets import Audio

dataset = dataset.cast_column(
    "audio",
    Audio(sampling_rate = 48_000)
)

In [ ]:
idx = 6
sample = dataset[idx]
sample["audio"]["sampling_rate"]
#audio sample ready for the model

48000

In [ ]:
# calp model takes audio and text as inpuit and computes similarity between the two
# text and audio correlate with each other: high similarity score as the output

labels = [
          "sound of a door knock",
          "sound of a dog",
          "sound of someone knocking"]

zs_classifier(sample["audio"]["array"], candidate_labels = labels)

[{'score': 0.7368903160095215, 'label': 'sound of someone knocking'},
 {'score': 0.2630600035190582, 'label': 'sound of a door knock'},
 {'score': 4.972614988218993e-05, 'label': 'sound of a dog'}]

In [ ]:
sample

{'filename': '1-101336-A-30.wav',
 'fold': 1,
 'target': 30,
 'category': 'door_wood_knock',
 'esc10': False,
 'src_file': 101336,
 'take': 'A',
 'audio': {'path': None,
  'array': array([1.16935509e-04, 1.92917010e-04, 8.99410225e-05, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]),
  'sampling_rate': 48000}}

## Automatic speech recognition: transcribing audio into text

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.8 MB/

In [ ]:
#wishper model by open-ai
#librispeech dataset corpus of 1000 hours of data from audio books
#streming = True: examples loaded one at a time

dataset = load_dataset("librispeech_asr",
                       split = "train.clean.100",
                       streaming = True)
#streamed datasets take examples one by one as audio data is huge to load

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for librispeech_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/librispeech_asr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
example = next(iter(dataset))
example

{'file': '374-180298-0000.flac',
 'audio': {'path': '374-180298-0000.flac',
  'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
         -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED',
 'speaker_id': 374,
 'chapter_id': 180298,
 'id': '374-180298-0000'}

In [ ]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(example["audio"]["array"], rate = example["audio"]["sampling_rate"])

In [ ]:
#first 5 examples

dataset_first_five = dataset.take(5)
list(dataset_first_five)

[{'file': '374-180298-0000.flac',
  'audio': {'path': '374-180298-0000.flac',
   'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
          -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
   'sampling_rate': 16000},
  'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED',
  'speaker_id': 374,
  'chapter_id': 180298,
  'id': '374-180298-0000'},
 {'file': '374-180298-0001.flac',
  'audio': {'path': '374-180298-0001.flac',
   'array': array([-9.15527344e-05, -1.52587891e-04, -1.52587891e-04, ...,
          -2.13623047e-04, -1.83105469e-04, -2.74658203e-04]),
   'sampling_rate': 16000},
  'text': "MARGUERITE TO BE UNABLE TO LIVE APART FROM ME IT WAS THE DAY AFTER THE EVENING WHEN SHE CAME TO SEE ME THAT I SENT HER MANON LESCAUT FROM THAT TIME SEEING THAT I COULD NOT CHANGE MY MISTRESS'S LIFE I CHANGED MY OWN",
  'speaker_id': 374

In [ ]:
example = list(dataset_first_five)[0]
print(f"Text: {example['text']}")
IPythonAudio(example["audio"]["array"], rate = example["audio"]["sampling_rate"])

Text: CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED


In [ ]:
##loading model
from transformers import pipeline

asr_model = pipeline(task = "automatic-speech-recognition",
                     model = "distil-whisper/distil-small.en")
#distil model is small model trained by responses of the the original model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/332M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [ ]:
dataset_sr = example['audio']['sampling_rate']
model_sr = asr_model.feature_extractor.sampling_rate
print(f'model sampling rate: {model_sr} dataset sampling rate: {dataset_sr}')

model sampling rate: 16000 dataset sampling rate: 16000


In [ ]:
##passing audio to the model
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_output = asr_model(example['audio']['array'])
model_output

{'text': ' Chapter 16 I might have told you of the beginning of this liaison in a few lines, but I wanted you to see every step by which we came. I too agree to whatever Marguerite wished.'}

In [ ]:
##comparing model output transcription to the label text

example['text']

'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED'

Gradio application

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
demo = gr.Blocks()

In [ ]:
def main(filepath):  #main function to transcribe speech given the file path
  if filepath is None:
    gr.Warning('No audio found, retry.')
    return ""
  output = asr_model(filepath)
  return output["text"]

In [ ]:
microphone_transcribe = gr.Interface(
    fn = main,
    inputs = gr.Audio(sources="microphone",
                    type="filepath"),
    outputs = gr.Textbox(label="Transcription", lines = 3),
    allow_flagging = "never",
)

In [ ]:
file_transcribe = gr.Interface(
    fn = main,
    inputs = gr.Audio(sources="upload",
                    type="filepath"),
    outputs = gr.Textbox(label="Transcription", lines = 3),
    allow_flagging = "never",
)

In [ ]:
with demo:
  gr.TabbedInterface(
      [microphone_transcribe, file_transcribe],
      ["Transcribe using Microphone", "Transcribe using Audio file"]
  )

demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4c9238ba214e484ad0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4c9238ba214e484ad0.gradio.live
